In [2]:
import torch
import torch.nn as nn
import numpy as np

### <u>Compare functions one by one</u>

#### 1. MaxPool1d

In [3]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.3731, 0.9901, 0.3004, 0.9839],
        [0.1905, 0.2237, 0.6336, 0.2609],
        [0.4826, 0.7048, 0.6619, 0.7387]])

In [4]:
#define the pytorch equivalent
pytorch_max_pool_fn = nn.MaxPool1d(kernel_size=2, stride=1, padding=0,dilation=1, return_indices=False, ceil_mode=False)
output_from_pytorch = pytorch_max_pool_fn(random_tensor)
output_from_pytorch

tensor([[0.9901, 0.9901, 0.9839],
        [0.2237, 0.6336, 0.6336],
        [0.7048, 0.7048, 0.7387]])

In [5]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_MaxPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(max(row[k:k + kernel_size]))
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_MaxPool1d(random_tensor)
output_from_my_implementation

tensor([[0.9901, 0.9901, 0.9839],
        [0.2237, 0.6336, 0.6336],
        [0.7048, 0.7048, 0.7387]])

In [6]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 2. AvgPool1d

In [7]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.0499, 0.1304, 0.0914, 0.5263],
        [0.3252, 0.3168, 0.3161, 0.8598],
        [0.2753, 0.9019, 0.5978, 0.6983]])

In [8]:
#define the pytorch equivalent
pytorch_avg_pool_fn = nn.AvgPool1d(kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True)
output_from_pytorch = pytorch_avg_pool_fn(random_tensor)
output_from_pytorch


tensor([[0.0901, 0.1109, 0.3089],
        [0.3210, 0.3165, 0.5880],
        [0.5886, 0.7499, 0.6481]])

In [9]:
#define my custom implementation of avgPool1d ( assumes that x is 2D (C,W) shape)
def my_AvgPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(sum(row[k:k + kernel_size])/kernel_size)
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_AvgPool1d(random_tensor)
output_from_my_implementation

tensor([[0.0901, 0.1109, 0.3089],
        [0.3210, 0.3165, 0.5880],
        [0.5886, 0.7499, 0.6481]])

In [10]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 3. Conv1d

In [11]:
#Loading the given filter and input tensors
pixel_input = torch.load("./pixel_input.pt") #Shape (1,1,32)
filter = torch.load("./4_3_filter.pt") #(Shape 3,1,1)

In [15]:
#define pytorch equivalent
pytorch_conv1d_output = nn.functional.conv1d(pixel_input, filter, bias = None,stride = 1, padding = 0, dilation = 1, groups = 1)

#Shape is (1,3,32)

pytorch_conv1d_output

tensor([[[0.2125, 0.3250, 0.4375, 0.5500, 0.6625, 0.7750, 0.8875, 1.0000,
          1.1125, 1.2250, 1.3375, 1.4500, 1.5625, 1.6750, 1.7500, 1.7625,
          1.7000, 1.5875, 1.4750, 1.3625, 1.2500, 1.1375, 1.0250, 0.9125,
          0.8000, 0.6875, 0.5750, 0.4625, 0.3500, 0.2375]]])

In [16]:
#define custom implementation
def my_Conv1d(inp, filter, stride=1, padding=0, dilation=1, groups = 1):
    result = []

    for f in filter:
        #as f in this case is 1x1 ( but it could in theory be something like 1 x k)
        f = f[0]
        batch_row = []
        for batch in inp: #(1,32)
            conv_row = []
            for channel in batch: #(32,)
                #convolve f over this channel array using this filter by using dot product to convolve!
                for f_ind in range(0, len(channel) - len(f) + 1, stride):
                    f = np.array(f)
                    conv_row.append(np.dot(f, np.array(channel[f_ind: f_ind + len(f)])))
            batch_row.append(conv_row)
        result.append(batch_row)

    result = torch.tensor(result)

    #permute it so that the shapes are correct.
    result = result.permute(1,0,2)
    return result

my_conv1d_output = my_Conv1d(pixel_input, filter)
my_conv1d_output

tensor([[[0.2125, 0.3250, 0.4375, 0.5500, 0.6625, 0.7750, 0.8875, 1.0000,
          1.1125, 1.2250, 1.3375, 1.4500, 1.5625, 1.6750, 1.7500, 1.7625,
          1.7000, 1.5875, 1.4750, 1.3625, 1.2500, 1.1375, 1.0250, 0.9125,
          0.8000, 0.6875, 0.5750, 0.4625, 0.3500, 0.2375]]])

In [18]:
#Assert that both are the same
torch.allclose(output_from_pytorch, output_from_my_implementation, rtol=1e-05, atol=1e-05)

True

#### 4. Sigmoid



In [122]:
#Create some random input
random_tensor = torch.rand(2, 2)
random_tensor

tensor([[0.8103, 0.1572],
        [0.4445, 0.2277]])

In [123]:
#define the pytorch equivalent
pytorch_sigmoid_fn = nn.Sigmoid()
output_from_pytorch = pytorch_sigmoid_fn(random_tensor)
output_from_pytorch


tensor([[0.6922, 0.5392],
        [0.6093, 0.5567]])

In [124]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_Sigmoid(x):
    return 1/(1+np.exp(-x))

output_from_my_implementation = torch.tensor(my_Sigmoid(random_tensor))
output_from_my_implementation

C:\Users\paras\AppData\Local\Temp\ipykernel_5716\1711964653.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_from_my_implementation = torch.tensor(my_Sigmoid(random_tensor))


tensor([[0.6922, 0.5392],
        [0.6093, 0.5567]])

In [125]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 5. BatchNorm1d

In [126]:
#Create some random input
random_tensor = torch.rand(2,1,4) #(batchsize,numfeatures, actualfeatures)
random_tensor

tensor([[[0.8012, 0.8653, 0.0883, 0.3625]],

        [[0.9214, 0.6491, 0.8451, 0.5259]]])

In [127]:
#define the pytorch equivalent
pytorch_batchnorm_fn = nn.BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
output_from_pytorch = pytorch_batchnorm_fn(random_tensor)
output_from_pytorch = output_from_pytorch.data.detach()
output_from_pytorch

tensor([[[ 0.6205,  0.8562, -1.9999, -0.9918]],

        [[ 1.0624,  0.0616,  0.7822, -0.3914]]])

In [128]:
#define my custom implementation of batchnorm ( assumes that x is 3D (B,1,N) shape)
def my_BatchNorm1d(x, eps=1e-05, momentum=0.1, affine=True):
    #momentum is used only during training and not used during inference!
    # Compute batch mean and batch variance
    batch_mean = x.mean(dim=(0, 2), keepdim=True)
    batch_var = x.var(dim=(0, 2), unbiased=False, keepdim=True)

    # Normalize the input using batch mean and variance
    x_normalized = (x - batch_mean) / torch.sqrt(batch_var + eps)

    return x_normalized

output_from_my_implementation = my_BatchNorm1d(random_tensor)
output_from_my_implementation

tensor([[[ 0.6205,  0.8562, -1.9999, -0.9918]],

        [[ 1.0624,  0.0616,  0.7822, -0.3914]]])

In [129]:
#Assert that both are the same
torch.allclose(output_from_pytorch, output_from_my_implementation, rtol=1e-05, atol=1e-05)

True

#### 6. Linear

In [24]:
#Create some random input
weight = torch.load("./4_6_weight.pt") #Shape (1,1,32)
bias = torch.load("./4_6_bias.pt") #(Shape 3,1,1)

random_tensor = torch.rand(16,32)

In [25]:
weight.size()

torch.Size([16, 32])

In [26]:
bias.size()

torch.Size([16])

In [27]:
#define the pytorch equivalent
in_features = 32
out_features = 16
pytorch_linear_fn = nn.Linear(in_features, out_features, bias=True)

pytorch_linear_fn.weight = nn.Parameter(weight)
pytorch_linear_fn.bias = nn.Parameter(bias)

output_from_pytorch = pytorch_linear_fn(random_tensor)
output_from_pytorch = output_from_pytorch.data.detach()
output_from_pytorch

tensor([[-4.5783e-01,  2.3023e-01, -7.9213e-02, -3.6524e-01,  3.6225e-01,
          3.5586e-01,  1.8898e-01, -2.4559e-01, -1.1320e-01, -1.2822e-01,
          1.6586e-01, -2.1026e-01,  2.1441e-01,  1.1249e-01,  5.5575e-01,
          1.2107e-01],
        [-4.7706e-01, -7.8368e-04, -2.0029e-01, -1.3794e-01,  1.6647e-01,
          3.6221e-01,  6.5435e-02, -9.4932e-02,  6.3805e-02, -3.0431e-01,
          3.0784e-01, -5.0099e-02,  2.2768e-01,  4.9945e-02,  6.4397e-01,
         -1.4372e-01],
        [-2.8248e-01,  3.3805e-01,  2.7837e-02, -1.0285e-01,  1.5755e-01,
          6.7600e-01,  1.6041e-01, -1.7669e-01,  3.8766e-02, -3.3473e-01,
          4.4163e-01, -2.8571e-01,  2.2416e-01,  5.7681e-03,  7.9890e-01,
         -2.5715e-01],
        [-5.1823e-01,  3.1956e-01,  5.3780e-02, -2.9436e-01,  1.2818e-01,
          6.1528e-01, -1.3541e-02, -1.2519e-02, -1.7497e-03, -2.3324e-01,
          8.7738e-02, -6.2706e-02, -6.3785e-02, -4.1430e-01,  7.7243e-01,
         -2.1140e-01],
        [-4.7259e-01

In [28]:
#define my custom implementation of batchnorm ( assumes that x is 3D (B,1,N) shape)
def my_Linear(x, in_features=32, out_features=16):
    weights = torch.randn(in_features, out_features)  # Corrected order
    biases = torch.randn(out_features)

    # Perform linear transformation
    return torch.tensor(np.matmul(x, weights)) + biases

output_from_my_implementation = my_Linear(random_tensor)
output_from_my_implementation

C:\Users\paras\AppData\Local\Temp\ipykernel_7592\1457816834.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(np.matmul(x, weights)) + biases


tensor([[-0.0426,  2.6996,  4.5106,  0.8164, -1.3926,  2.1177, -1.5440, -2.8177,
          5.0069, -0.3544, -3.5589, -0.1442,  2.6068, -0.4582,  5.5969,  0.4362],
        [ 2.1838,  2.9080,  5.6041,  0.3362, -6.7615, -0.8284,  3.1849, -3.8562,
          4.3269, -0.0338, -5.0396,  0.6504,  3.4249,  0.3185,  4.0469, -2.3122],
        [ 3.6485,  2.3772,  5.6710,  1.0802, -3.8698,  1.9853,  3.9599, -4.1320,
          7.1967, -3.3152, -9.1698, -2.8233,  2.9046,  0.4814,  5.2547, -2.7222],
        [ 1.7515,  3.5249,  7.1651,  4.8252, -3.8191,  0.1266,  1.8229, -4.9273,
          7.0129, -4.6480, -6.7361, -0.9563,  8.0241, -1.7583,  5.1491, -2.2574],
        [ 1.4616,  3.3655,  5.8384,  2.2593, -6.4336,  0.3365,  1.8535, -5.5435,
          6.9341, -2.6563, -7.2061, -1.9507,  4.8486, -1.6909,  5.2024,  0.3090],
        [ 1.4739,  1.2487,  3.7316,  2.6677, -3.0271, -0.1294,  1.4957, -2.0452,
          3.6811, -2.4156, -4.1982, -3.0536,  1.8146, -2.7537,  5.4162, -0.3332],
        [ 3.2257,  5.8

In [29]:
#Assert that both are the same size as the data inside may be random
output_from_pytorch.shape == output_from_my_implementation.shape

True